In [ ]:
import time
import requests
import json
headers = {
    'accept': '*/*',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'cache-control': 'no-cache',
    'origin': 'https://solana.org',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://solana.org/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
}

def fetch_validators(offset, limit):
    params = {
        'offset': str(offset),
        'limit': str(limit),
    }
    response = requests.get('https://api.solana.org/api/validators/list', params=params, headers=headers)
    return response.json()['data']

all_validators = []
offset = 0
limit = 40

while True:
    validators = fetch_validators(offset, limit)
    if len(validators) == 0:
        break
    print(f"Fetched {len(validators)} validators, offset: {offset}")
    all_validators.extend(validators)
    offset += limit
    time.sleep(1)
# Save the data to a JSON file
with open('data/sfdp_data.json', 'w') as f:
    json.dump(all_validators, f, indent=2)

print(f"Scraped {len(all_validators)} validators and saved to data/sfdp_data.json")


In [2]:
import time
import json
vote_account_to_authority = {}
with open("data/627.json", "r") as f:
    rows = json.load(f)
import requests
import json
url = "https://mainnet.helius-rpc.com/?api-key=b77a1206-0e10-4f0b-a5b3-1641dc5021bd"
headers = {"Content-Type": "application/json"}

def fetch_vote_accounts(pubkey):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getVoteAccounts",
        "params": [
            {
                "votePubkey": pubkey
            }
        ]
    }
    
    retries = 10 
    for attempt in range(retries):
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            
            if 'result' in data:
                if 'current' in data['result'] and len(data['result']['current']) > 0:
                    vote_account = data['result']['current'][0]
                    return vote_account['nodePubkey'], "current"
                elif 'delinquent' in data['result'] and len(data['result']['delinquent']) > 0:
                    vote_account = data['result']['delinquent'][0]
                    return vote_account['nodePubkey'], "delinquent"
                else:
                    print(data)
                    return None
            else:
                print(data)
                return None
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == retries - 1:
                raise
            time.sleep(2)

from concurrent.futures import ThreadPoolExecutor, as_completed

def process_row(i, row):
    pubkey = row[0]
    vote_account = fetch_vote_accounts(pubkey)
    vote_account_to_authority[pubkey] = vote_account
    print(f"{i}/{len(rows)} Found vote account for {pubkey} {vote_account}")

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(process_row, i, row) for i, row in enumerate(rows)]
    for future in as_completed(futures):
        future.result()

print(f"Fetched {len(vote_account_to_authority)} vote account to authority mappings")


2/2685 Found vote account for 8ZfSRfAuDLV86PaKSCNfgx3AU1w6Hp8sGrEgFTkHViCt ('8rWDbEsuz4UWF2ZXiHhMECPkTZm51YrRaQSoSz8RPz2H', 'current')0/2685 Found vote account for 3Rk99suwAvvJgyLpDYEJeC2YPPLw1enc5T1r6J8ZoRSr ('6aiX7kVpUovCpbrLsMzG92qHcyhBrFcviDWHn2VzYYGB', 'current')
1/2685 Found vote account for Dca6LSCR24veJsdLbDDS8n73vvNfSdQaa4mXhVjTXUWD ('BD63LnAhKdHgGw2jJAm8gnfDTBMhaUpC86eAAc6NHNyk', 'delinquent')

3/2685 Found vote account for FMKT6kHBkmPf3LjZV5tpo3oR4m32rF8nF8JtC3WWcWoN ('E3HNUGWG9mggTLuxHDTY3sAortjTJTU7SwwEkjwV9REB', 'current')
4/2685 Found vote account for AnodeNCzJGQ7QwheFqJr6EknKGa72m6XHiLtDQiXcmEc ('6Rk694kh1QTyQkirdb1uDZmS5xqG9bNaYtxx8d311Mr7', 'current')
5/2685 Found vote account for 5szskKdH8nfnUuHTvn9hnhH3Xuvo7RVmcDDvD5WD7yNh ('Ee8dX3qtwrDRnxYK6NGQfmMeKT3Qpp2QZHpxiAiw23W9', 'current')
Attempt 1 failed: 429 Client Error: Too Many Requests for url: https://mainnet.helius-rpc.com/?api-key=b77a1206-0e10-4f0b-a5b3-1641dc5021bd
8/2685 Found vote account for 6ZUCygEHeA1MRgdxZ

In [10]:
import json

# Dump the vote_account_to_authority map to a file
filtered_vote_account_to_authority = {k: v for k, v in vote_account_to_authority.items() if v is not None}
with open('data/vote_account_to_authority.json', 'w') as f:
    json.dump(filtered_vote_account_to_authority, f)


In [28]:
# Cross reference stakeview.app data with the authority pubkeys
vote_account_to_authority = json.load(open('data/vote_account_to_authority.json'))
sfdp = {}
data = []

# Init the Authority -> SFDP data map
with open("data/sfdp_data.json", "r") as f:
    data = json.load(f)
    for row in data:
        key = row['mainnetBetaPubkey']
        sfdp[key] = row

lost_stake = []
with open(f'data/627.json', 'r') as f:
    rows = json.load(f)
    for row in rows:
        pubkey = row[0]
        authority_pubkey, status = vote_account_to_authority.get(pubkey, (None, None))
        details = sfdp.get(authority_pubkey, {})
        mnName = details.get('mnName', "")
        if row[1]['stake'] and details:
            stake_delta = row[1]['stake'].get('delta_lamports', None)
            if stake_delta:
                stake_delta = float(stake_delta) / 1e9
                if stake_delta < 0:
                    lost_stake.append((pubkey, mnName, stake_delta))

lost_stake.sort(key=lambda x: x[2])  # Sort lost_stake by stake_delta
print('\n'.join([f"{x[0]} {x[1]} {x[2]}" for x in lost_stake[0:100]]))

AE6xdVD5e92ZgevjWAamoF5kuAu88AvmUv4RRdBcoyz3 SolCandy🍬 +MEV -130418.019577461
Eajfs6oXGGkvjYsxkQZZJcDCLLkUajaHizfgg2xTsqyd JM -129855.759161099
2qsJLygBZ2XoYkRtkngH4fH4CtFmzfnjARbtMSWkZAQs None -129746.657942587
9w3YAf77BSGLQXahTpwAe9U9mNdCXkqmxH17uCyN1wyH lelika -129666.314764921
3r5ZXC1yFqMmk8VwDdUJbEdPmZ8KZvEkzd5ThEYRetTk Vnode -129654.824452454
We11J5D4iXcNbdMwCZX2o9RRkwaWBo1AGLADfubmeTb ✌❤☯ WellDoneStake -129551.860968602
BJ6RVC6Moc9mB8u3qcubnhS9ejninfPfXvz6kFGgRH8v Valdorbro -123911.639655904
HsUhGC6yKwr8kLC5gdqVLrDfKsGCd2ZAXiabP1WojuzA None -123197.325311893
GB44NXtM7zGm6QnzQjzHZcRKSswkJbox8aJsKiXGbFJr Rustiq Technology -122775.586630417
HQuUQmerqwvBRFo1moWgNPpcc43EJxGUQZmgrrmqA9sA Let's Node -115437.626935405
Hi8vxcjvAoLSDB9a9xjreJBFurMg41UQhNndx3qS1BJW Geleon -110234.178444871
yrfQfUfsZechz1zqQyTRRz43czTZQidcrm4SNVWiDPi romanh -102635.758242973
CT2CzbiNRz8ccgWQZR4BN7cpm3rDyWyQxUf5MNbMom7n MasterYarik -97837.585832549
5ni6KoVM62cRJNfFFKGdiyDfYbKWWAGZ21cfGZcj1y66 Custodian 💼 0% 

In [22]:
import pandas as pd
df = pd.DataFrame(data, columns=['pubkey', 'epoch', 'delta', 'active', 'activating', 'deactivating', 'isSFDP'])

df.head()

,pubkey,epoch,delta,active,activating,deactivating,isSFDP
0,6SmEcnuXJ3ZYqJWTqCoPQMhfvey6PfWb1LebLzoH3f7m,623,148836401864,235245445975626,444788463179,195305645162,False
1,phz34EcgWRCT9otPzRS2JtSzVHxQJk4SovqJvV1TQk8,623,-267939089863,60203846760309,4904044035804,100938115236,False
2,D4A3ZYhd3zH6h8QKykreea7x3ajvuwnw1yZD3nRrjiHC,623,-986318398306,29948904862468,340761247106,0,False
3,7YzmMGQhrE7VaTiDFnb7R633gquLEKmSpwkXG6u64fhj,623,-277568586277,29950122840004,340762724106,0,False
4,A5dLWA1kcZZSEjrphF814Da7PW98gz72eG9EPTR4wuqb,623,5799676921302,24920452733757,2251355403903,0,False


In [23]:
df[df['isSFDP'] == True]

,pubkey,epoch,delta,active,activating,deactivating,isSFDP
